# 嵌入模型微调

👏🏻 欢迎来到 uniem 的微调教程，在这里您将学习到如何使用 uniem 库提供的 `FineTuner` 对 M3E 等模型进行微调。如果您是在 colab 环境中运行，请使用 GPU 运行时。

In [ ]:
# 最开始肯定是安装 `uniem` 库了 😉
!pip install uniem

`uniem` 已经安装完成了，让先看一个简单的例子，来感受一下微调的过程。

In [ ]:
from datasets import load_dataset
from uniem.finetuner import FineTuner

dataset = load_dataset('shibing624/nli_zh', 'STS-B')
finetuner = FineTuner.from_pretrained('moka-ai/m3e-small', dataset=dataset)
finetuner.run(epochs=3)

🎉 微调已经完成了，通过 `FineTuner` 我们只需要几行代码就可以完成微调，就像魔法一样！

让我们看看这背后发生了什么，为什么可以这么简单？

1. `FineTuner` 会自动加载 M3E 模型，您只需要声明即可，就像例子中的 `moka-ai/m3e-small`
2. `FineTuner` 会自动识别数据格式，只要您的数据类型在 `FineTuner` 支持的范围内，`FineTuner` 就会自动识别并加以使用
3. `FineTuner` 会自动选择训练方式，`FineTuner` 会根据模型和数据集自动地选择训练方式，即 对比学习 或者 CoSent 等
4. `FineTuner` 会自动选择训练环境和超参数，`FineTuner` 会根据您的硬件环境自动选择训练设备，并根据模型、数据等各种信息自动建议最佳的超参数，lr, batch_size 等，当然您也可以自己手动进行调整
5. `FineTuner` 会自动保存微调记录和模型，`FineTuner` 会根据您的设置自动使用您环境中的 wandb, tensorboard 等来记录微调过程，同时也会自动保存微调模型

总结一下，`FineTuner` 会自动完成微调所需的各种工作，只要您的数据类型在 `FineTuner` 支持的范围内！

那么，让我们看看 `FineTuner` 都支持哪些类型的数据吧。

## FineTuner 支持的数据类型

`FineTuner` 中 `dataset` 参数是一个可供迭代 (for 循环) 的数据集，每次迭代会返回一个样本，这个样本应该是以下三种格式之一：

1. `PairRecord`，句对样本
2. `TripletRecord`，句子三元组样本
3. `ScoredPairRecord`，带有分数的句对样本

In [ ]:
import os
import warnings
from uniem.data_structures import RecordType, PairRecord, TripletRecord, ScoredPairRecord

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
warnings.filterwarnings('ignore')
print(f'record_types: {[record_type.value for record_type in RecordType]}')

### PairRecord

`PairRecord` 就是句对样本，每一个样本都代表一对相似的句子，字段的名称是 `text` 和 `text_pos`

In [ ]:
pair_record = PairRecord(text='肾结石如何治疗？', text_pos='如何治愈肾结石')
print(f'pair_record: {pair_record}')

### TripletRecord

`TripletRecord` 就是句子三元组样本，在 `PairRecord` 的基础上增加了一个不相似句子负例，字段的名称是 `text`、`text_pos` 和 `text_neg`

In [ ]:
triplet_record = TripletRecord(text='肾结石如何治疗？', text_pos='如何治愈肾结石', text_neg='胆结石有哪些治疗方法？')
print(f'triplet_record: {triplet_record}')

### ScoredPairRecord

`ScoredPairRecord` 就是带有分数的句对样本，在 `PairRecord` 的基础上添加了句对的相似分数(程度)。字段的名称是 `sentence1` 和 `sentence2`，以及 `label`。

In [ ]:
# 1.0 代表相似，0.0 代表不相似
scored_pair_record1 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='如何治愈肾结石', label=1.0)
scored_pair_record2 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='胆结石有哪些治疗方法？', label=0.0)
print(f'scored_pair_record: {scored_pair_record1}')
print(f'scored_pair_record: {scored_pair_record2}')

In [ ]:
# 2.0 代表相似，1.0 代表部分相似，0.0 代表不相似
scored_pair_record1 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='如何治愈肾结石', label=2.0)
scored_pair_record2 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='胆结石有哪些治疗方法？', label=1.0)
scored_pair_record3 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='失眠如何治疗', label=0)
print(f'scored_pair_record: {scored_pair_record1}')
print(f'scored_pair_record: {scored_pair_record2}')
print(f'scored_pair_record: {scored_pair_record3}')

#### 小结

`FineTuner` 支持的数据类型有三种，分别是 `PairRecord`，`TripletRecord` 和 `ScoredPairRecord`，其中 `TripletRecord` 比 `PairRecord` 多了一个不相似句子负例，而 `ScoredPairRecord` 是在 `PairRecord` 的基础上添加了句对的相似分数。

只要您的数据集是这三种类型之一，`FineTuner` 就可以自动识别并使用。现在让我们看看实际的例子

## 示例：医疗相似问题

现在我们假设我们想要 HuggingFace 上托管的 vegaviazhang/Med_QQpairs 医疗数据集上做微调，让我们先把数据集下载好

In [ ]:
from datasets import load_dataset

med_dataset_dict = load_dataset('vegaviazhang/Med_QQpairs')

让我们查看一下 Med_QQpairs 的数据格式是不是在 `FineTuner` 支持的范围内

In [ ]:
print(med_dataset_dict['train'][0])
print(med_dataset_dict['train'][1])

我们发现 Med_QQpairs 数据集正好符合我们 `ScoredPairRecord` 的数据格式，只是字段名称是 `question1` 和 `question2`，我们只需要修改成 `sentence1` 和 `sentence2` 就可以直接进行微调了

In [ ]:
from datasets import load_dataset

from uniem.finetuner import FineTuner

dataset = load_dataset('vegaviazhang/Med_QQpairs')
dataset = dataset.rename_columns({'question1': 'sentence1', 'question2': 'sentence2'})
# 指定训练的模型为 m3e-small
finetuner = FineTuner.from_pretrained('moka-ai/m3e-small', dataset=dataset)
fintuned_model = finetuner.run(epochs=3)

训练过程完成后，会自动保存模型到 finetuned-model 目录下

In [ ]:
!ls finetuned-model/model

## 示例：猜谜

现在我们要对一个猜谜的数据集进行微调，这个数据集是通过 json line 的形式存储的，让我先看看数据格式吧。

In [ ]:
import pandas as pd

df = pd.read_json('https://raw.githubusercontent.com/wangyuxinwhy/uniem/main/examples/example_data/riddle.jsonl', lines=True)
records = df.to_dict('records')
print(records[0])
print(records[1])

这个数据集中，我们有 `instruction` 和 `output` ，我们可以把这两者看成一个相似句对。这是一个典型的 `PairRecord` 数据集。

`PairRecord` 需要 `text` 和 `text_pos` 两个字段，因此我们需要对数据集的字段进行重新命名，以符合 `PairRecord` 的格式。

In [ ]:
import pandas as pd

from uniem.finetuner import FineTuner

# 读取 jsonl 文件
df = pd.read_json('https://raw.githubusercontent.com/wangyuxinwhy/uniem/main/examples/example_data/riddle.jsonl', lines=True)
# 重新命名
df = df.rename(columns={'instruction': 'text', 'output': 'text_pos'})
# 指定训练的模型为 m3e-small
finetuner = FineTuner.from_pretrained('moka-ai/m3e-small', dataset=df.to_dict('records'))
fintuned_model = finetuner.run(epochs=3, output_dir='finetuned-model-riddle')

上面的两个示例分别展示了对 jsonl 本地 `PairRecord` 类型数据集，以及 huggingface 远程 `ScoredPair` 类型数据集的读取和训练过程。`TripletRecord` 类型的数据集的读取和训练过程与 `PairRecord` 类型的数据集的读取和训练过程类似，这里就不再赘述了。

也就是说，你只要构造了符合 `uniem` 支持的数据格式的数据集，就可以使用 `FineTuner` 对你的模型进行微调了。

`FineTuner` 接受的 dataset 参数，只要是可以迭代的产生有指定格式的字典 `dict` 就行了，所以上述示例分别使用 `datasets.DatasetDict` 和 `list[dict]` 两种数据格式。

## 示例：sentence-transformers/all-MiniLM-L6-v2

`FineTuner` 在设计实现的时候也同时兼容了其他框架的模型，而不仅仅是 uniem！比如， `sentece_transformers` 的 [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) 是一个广受欢迎的模型。现在我们将使用前文提到过的 Med_QQpairs 对其进行微调。

In [ ]:
from datasets import load_dataset

from uniem.finetuner import FineTuner

dataset = load_dataset('vegaviazhang/Med_QQpairs')
dataset = dataset.rename_columns({'question1': 'sentence1', 'question2': 'sentence2'})
# model_name 为 sentence-transformers/all-MiniLM-L6-v2
# model_type 为 sentence_transformers
finetuner = FineTuner.from_pretrained('sentence-transformers/all-MiniLM-L6-v2', dataset=dataset, model_type='sentence_transformers')
fintuned_model = finetuner.run(epochs=3, batch_size=32)

## 示例：从预训练模型开始训练

我们除了可以在 M3E 的基础上进行微调之外，还可以选择从一个预训练模型开始训练，这个预训练模型可以是 BERT，RoBERTa，T5 等。另外，我们将使用迭代式的数据集，通过 datasets 的 streaming 的方式来使用一个较大的数据集，并对一个只有两层的 BERT `uer/chinese_roberta_L-2_H-128` 进行微调。

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

from uniem.finetuner import FineTuner
from uniem.model import create_uniem_embedder

dataset = load_dataset('shibing624/nli-zh-all', streaming=True)
dataset = dataset.rename_columns({'text1': 'sentence1', 'text2': 'sentence2'})
embedder = create_uniem_embedder('uer/chinese_roberta_L-2_H-128')
tokenizer = AutoTokenizer.from_pretrained('uer/chinese_roberta_L-2_H-128')
finetuner = FineTuner(embedder, tokenizer=tokenizer, dataset=dataset)
fintuned_model = finetuner.run(epochs=3, batch_size=32, lr=1e-3)

/Users/wangyuxin/opt/miniconda3/envs/uniem/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at uer/chinese_roberta_L-2_H-128 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly i

Batch size: 32
Start with seed: 42
Output dir: finetuned-model
Disable shuffle for iterable dataset
Learning rate: 0.0001
Start training for 3 epochs


Epoch 1/3 - loss: 4.1468: : 1329it [04:21,  5.09it/s]
Epoch 2/3 - loss: 4.6541: : 905it [01:34,  8.46it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 embedder = create_uniem_embedder('uer/chinese_roberta_L-2_H-128')                           │
│   10 tokenizer = AutoTokenizer.from_pretrained('uer/chinese_roberta_L-2_H-128')                  │
│   11 finetuner = FineTuner(embedder, tokenizer=tokenizer, dataset=dataset)                       │
│ ❱ 12 fintuned_model = finetuner.run(epochs=3, batch_size=32)                                     │
│   13                                                                                             │
│                                                                                                  │
│ /Users/wangyuxin/workspace/githubs/uniem/uniem/utils.py:122 in decorator                         │
│                                                                                                  │
│   119 │   │   is_manually_passed_batch_size = 'batch_size' in kwargs                             │
│   120 │   │                                                                                      │
│   121 │   │   if is_manually_passed_batch_size:                                                  │
│ ❱ 122 │   │   │   return function(*args, **kwargs)                                               │
│   123 │   │   else:                                                                              │
│   124 │   │   │   while True:                                                                    │
│   125 │   │   │   │   if batch_size == 0:                                                        │
│                                                                                                  │
│ /Users/wangyuxin/workspace/githubs/uniem/uniem/finetuner.py:309 in run                           │
│                                                                                                  │
│   306 │   │   │   save_on_epoch_end=save_on_epoch_end,                                           │
│   307 │   │   )                                                                                  │
│   308 │   │   accelerator.print(f'Start training for {epochs} epochs')                           │
│ ❱ 309 │   │   trainer.train()                                                                    │
│   310 │   │                                                                                      │
│   311 │   │   accelerator.wait_for_everyone()                                                    │
│   312 │   │   accelerator.print('Training finished')                                             │
│                                                                                                  │
│ /Users/wangyuxin/workspace/githubs/uniem/uniem/trainer.py:53 in train                            │
│                                                                                                  │
│    50 │   │   │   self.model.train()                                                             │
│    51 │   │   │   self.progress_bar.on_epoch_start()                                             │
│    52 │   │   │                                                                                  │
│ ❱  53 │   │   │   for batch_index, batch in enumerate(self.train_dataloader):                    │
│    54 │   │   │   │   with self.accelerator.accumulate(self.model):                              │
│    55 │   │   │   │   │   self.optimizer.zero_grad()                                             │
│    56 │   │   │   │   │   batch_output = self.model(**batch)                                     │
│                                                                                                  │
│ /Users/wangyuxin/opt/miniconda3/envs/uniem/lib/python3.10/s

## 示例：SGPT

`FineTuner` 在设计实现的时候还提供了更多的灵活性，以 [SGPT](https://github.com/Muennighoff/sgpt) 为例，SGPT 和前面介绍的模型主要有以下三点不同：

1. SGPT 使用 GPT 系列模型（transformer decoder）作为 Embedding 模型的基础模型
2. Embedding 向量的提取策略不再是 LastMeanPolling ，而是根据 token position 来加权平均
3. 使用 bitfit 的微调策略，在微调时只对模型的 bias 进行更新

现在我们将效仿 SGPT 的训练策略，使用 Med_QQpairs 对 GPT2 进行微调。

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

from uniem.finetuner import FineTuner
from uniem.training_strategy import BitFitTrainging
from uniem.model import PoolingStrategy, create_uniem_embedder

dataset = load_dataset('vegaviazhang/Med_QQpairs')
dataset = dataset.rename_columns({'question1': 'sentence1', 'question2': 'sentence2'})
embedder = create_uniem_embedder('gpt2', pooling_strategy=PoolingStrategy.last_weighted)
tokenizer = AutoTokenizer.from_pretrained('gpt2')
finetuner = FineTuner(embedder, tokenizer, dataset=dataset)
finetuner.tokenizer.pad_token = finetuner.tokenizer.eos_token
finetuner.run(epochs=3, lr=1e-3, batch_size=32, training_strategy=BitFitTrainging())